# Accessing PostgreSQL
- <a href="https://pypi.org/project/psycopg2/" target="_blank">PostgreSQL driver on PyPI</a>
- <a href="https://www.psycopg.org/docs/" target="_blank">PostgreSQL driver documentation</a>

In [ ]:
# See https://www.psycopg.org/
import psycopg2, pandas as pd
import ibm_db
import ibm_db_dbi
import warnings
import os

postgres_metadata = {
    "database": """customerpointofsale""",
    "password": """IBMDem0s!""",
    "port": """5432""",
    "query_timeout": """300""",
    "host": """ibmreplication.cdm6lav53g2d.us-east-2.rds.amazonaws.com""",
    "ssl": """true""",
    "username": """demouser"""
}

# Demo user connection
demo_connection = psycopg2.connect(
    host=postgres_metadata['host'],
    port=postgres_metadata['port'],
    database=postgres_metadata['database'],
    user=postgres_metadata["username"],
    password=postgres_metadata['password'],
)
# Db2 connection
credentials = {
    'username': "cpdemo",
    'password': "C!oudP@k4DataDem0s",
    'database': 'BLUDB',
    'host': "db2w-ovqfeqq.us-south.db2w.cloud.ibm.com",
    'port': 50001
}
dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "SECURITY=ssl;"
    "UID={3};"
    "PWD={4};").format(credentials['database'], credentials['host'],
                       credentials['port'], credentials['username'],
                       credentials['password'])

conn = ibm_db.connect(dsn, "", "")
pconn = ibm_db_dbi.Connection(conn)

# Ignore warnings about the driver
warnings.filterwarnings("ignore") # one of "error", "ignore", "always", "default", "module", or "once"
project_id = os.environ['PROJECT_ID']

# table name to use
table = "transactions"

### Try SELECT with demo

In [ ]:
# Try SELECT to make sure the data was loaded
query = 'SELECT * FROM {}'.format(table)

data_df_1 = pd.read_sql_query(query, con=demo_connection)
print("number of records: {}".format(data_df_1.shape[0]))
data_df_1.head()

### Try INSERT

In [ ]:
values = [
    "2023","August", "17", "10350", "Matt Denham", "data and AI",
    "Ottawa", "CA", "99", "1", "99.99", "1234", project_id
]
SQL = 'INSERT INTO {} VALUES (DEFAULT, {})'.format(table, values)
SQL = SQL.replace("[", "").replace("]", "") # Remove brackets araound the list
print(SQL)
with demo_connection.cursor() as curs:
    try:
        curs.execute(SQL)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        demo_connection.rollback()
    demo_connection.commit()

In [ ]:
# Retrieve the row to make sure it was inserted
query = 'SELECT * FROM {} WHERE uid = \'{}\''.format(table, project_id)
pd.read_sql_query(query, con=demo_connection)

### See if the row made it to Db2

In [ ]:
stmt = """
  SELECT * FROM IDR.TRANSACTIONS 
  WHERE uid = '{}'
""".format(project_id)

data_pd = pd.read_sql(stmt, pconn)
data_pd.head()

### Close the database connections

In [ ]:
# Close the database connections
demo_connection.close()
pconn.close()
conn.close()